## Documentation

To read more about the SQL search API, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/sql-search-api.html).

![sql_search_api_docs](../images/sql_search_api_docs.png)

## Connect to ElasticSearch

In [2]:
from pprint import pprint
from elasticsearch import Elasticsearch

HOST = "http://localhost:9200"

es = Elasticsearch(hosts=HOST)
client_info = es.info()
print("Connected tp Elasticsearch!")
pprint(client_info.body)

Connected tp Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'iugjHCt8SwCWRVd35xnJ0A',
 'name': '5013781c82bc',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-02-05T22:10:57.067596412Z',
             'build_flavor': 'default',
             'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.12.0',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.17.2'}}


## Index data

In [3]:
import json

from pprint import pprint


INDEX = "my_index"

settings = {
    "index": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

es.indices.delete(index=INDEX, ignore_unavailable=True)
es.indices.create(index=INDEX, settings=settings)

operations = []
astronomy_documents = json.load(open("../data/astronomy.json"))

for document in astronomy_documents:
    operations.append({"index": {"_index": INDEX}})
    operations.append(document)

response = es.bulk(operations=operations)
pprint(response.body)

{'errors': False,
 'items': [{'index': {'_id': 'vKdCNpUBObNT95My2Z5C',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 1},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'vadCNpUBObNT95My2Z5C',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 1,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 1},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'vqdCNpUBObNT95My2Z5C',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 2,
                      '_shards': {'failed': 0, 'successful': 1, '

In [4]:
count = es.count(index=INDEX)
print("Number of documents in index: ", count["count"])

Number of documents in index:  10


## SQL search API

### 1. Simple select query

In [5]:
query = {
    "query": "SELECT title FROM my_index ORDER BY id LIMIT 5"
}

response = es.sql.query(body=query)
for row in response["rows"]:
    print(row)

['The Solar System']
['Black Holes']
['Galaxies']
['The Big Bang Theory']
['Exoplanets']


### 2. Change response format

In [6]:
query = {
    "query": "SELECT * FROM my_index"
}

response = es.sql.query(body=query, format="json")
print(response)

{'columns': [{'name': 'content', 'type': 'text'}, {'name': 'id', 'type': 'long'}, {'name': 'title', 'type': 'text'}], 'rows': [['The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.', 1, 'The Solar System'], ['A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.', 2, 'Black Holes'], ['Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.', 3, 'Galaxies'], ['The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.', 4, 'The Big Bang Theory'], ['Exoplanets, or extrasolar planets, are plane

In [7]:
query = {
    "query": "SELECT * FROM my_index"
}

response = es.sql.query(body=query, format="txt")
print(response)

                                                                                                        content                                                                                                         |      id       |         title         
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------------
The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.                                          |1              |The Solar System       
A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.                      |2           

In [8]:
query = {
    "query": "SELECT * FROM my_index"
}

response = es.sql.query(body=query, format="csv")
print(response)

content,id,title
"The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.",1,The Solar System
"A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.",2,Black Holes
"Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.",3,Galaxies
The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.,4,The Big Bang Theory
"Exoplanets, or extrasolar planets, are planets that exist outside our solar system. They vary greatly in size and composition and are often found using methods like the transit method a

In [10]:
query = {
    "query": "SELECT * FROM my_index"
}

response = es.sql.query(body=query, format="tsv")
print(response)

content	id	title
The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.	1	The Solar System
A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.	2	Black Holes
Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.	3	Galaxies
The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.	4	The Big Bang Theory
Exoplanets, or extrasolar planets, are planets that exist outside our solar system. They vary greatly in size and composition and are often found using methods like the transit method and radi

### 3. Filter results

In [9]:
query = {
    "query": "SELECT * FROM my_index"
}

response = es.sql.query(body=query, filter={
    "term": {
        "title.keyword": "Black Holes"
    }
})
print(response)

{'columns': [{'name': 'content', 'type': 'text'}, {'name': 'id', 'type': 'long'}, {'name': 'title', 'type': 'text'}], 'rows': [['A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.', 2, 'Black Holes']]}


/tmp/ipykernel_7967/1820827543.py:5: DeprecationWarning: Received 'filter' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.sql.query(body=query, filter={


### 4. Pagination

In [11]:
query = {
    "query": "SELECT * FROM my_index ORDER BY id DESC"
}

response = es.sql.query(
    body=query,
    format='json',
    fetch_size=5,
)
response.body

/tmp/ipykernel_7967/3586786142.py:5: DeprecationWarning: Received 'fetch_size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.sql.query(


{'columns': [{'name': 'content', 'type': 'text'},
  {'name': 'id', 'type': 'long'},
  {'name': 'title', 'type': 'text'}],
 'rows': [['Space exploration involves the use of space technology to explore outer space. It includes missions to planets, moons, and other celestial bodies, aiming to discover more about the universe.',
   10,
   'Space Exploration'],
  ['The universe has been expanding since the Big Bang. Observations of distant galaxies show that they are moving away from us, which supports the idea of an expanding universe.',
   9,
   'The Expanding Universe'],
  ["Dark matter is a type of matter that does not emit light or energy. It cannot be observed directly but is believed to make up about 27% of the universe's total mass and energy.",
   8,
   'Dark Matter'],
  ['Astrobiology is the study of the origin, evolution, distribution, and future of life in the universe. It combines elements of biology, chemistry, and planetary science.',
   7,
   'Astrobiology'],
  ['Stars are b

### 5. Translate API

In [13]:
translate_query = {
    "query": "SELECT * FROM my_index WHERE content LIKE '%universe' ORDER BY id DESC LIMIT 20"
}

translated_query = es.sql.translate(body=translate_query)
translated_query.body

{'size': 20,
 'query': {'wildcard': {'content.keyword': {'wildcard': '*universe',
    'boost': 1.0}}},
 '_source': False,
 'fields': [{'field': 'content'}, {'field': 'id'}, {'field': 'title'}],
 'sort': [{'id': {'order': 'desc',
    'missing': '_first',
    'unmapped_type': 'long'}}],
 'track_total_hits': -1}